In [1]:
from pyspark.sql import SparkSession

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("extract-cset-per-type") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("extract-cset-per-type").getOrCreate()

spark.conf.set("spark.worker.cleanup.enabled",True)
spark.conf.set("spark.worker.cleanup.interval",1800)
spark.conf.set("spark.worker.cleanup.appDataTtl",3600)
spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/29 23:50:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
%ls

2nouveauExemplePetitGraph.ipynb     coucou1/
APushExemplePetitGraph.ipynb        covavevolutionresult-fix.json/
AffichageDiagrammesDiapo.ipynb      cset-apres-per-type.zip
AffichageResultatsDiagramme.ipynb   cset-avant-per-type.zip
AveragePredicatesPetitGraphe.ipynb  intersection.ipynb
CScompleteWDC-test.ipynb            nouveauExemplePetitGraph.ipynb
CScompleteWDC.ipynb                 partialCSETtest.ipynb
CalculJSONTypes.ipynb               petitGrapheRDFapres.txt
ExemplePetitGraphe.ipynb            petitGrapheRDFavant.txt
JsonCsetFilesGenerator.ipynb        schemaTypes.json
Test-JsonCsetFilesGenerator.ipynb   spark-warehouse/
Untitled1.ipynb                     test-out/
WDCanalyseTemporelle.ipynb          test.json
coucou/                             zipData.ipynb


In [2]:
from pyspark.sql import functions as f

from pyspark.sql.functions import split
from pyspark.sql.functions import col

readavant = spark.read.option("header",True) \
  .csv("s3a://test-out/wdcfix/**")

csavant=readavant.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csavant.createOrReplaceTempView("CSET_avant")
csavant.show(truncate=150)

readapres = spark.read.option("header",True) \
  .csv("s3a://test-out/cset-wdc-2023-fix2/**")

# fix wrong formatting
csapres = readapres.withColumn("pset", f.regexp_replace(f.col("pset"), "isa:<([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?", "isa:<"))
csapres = csapres.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))


csapres.createOrReplaceTempView("CSET_apres")
csapres.show(truncate=150)

23/12/22 16:49:06 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                                                                                  pset|       count|
+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                     <schema.org/item> <schema.org/position> isa:<schema.org/ListItem>|6.82461002E8|
|                                                                                          <schema.org/itemListElement> isa:<schema.org/BreadcrumbList>|4.56168963E8|
|                                                                   <schema.org/item> <schema.org/name> <schema.org/position> isa:<schema.org/ListItem>| 3.9303227E8|
|   

+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                                                                                  pset|       count|
+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                     <schema.org/item> <schema.org/position> isa:<schema.org/ListItem>|6.69795521E8|
|                                                                   <schema.org/item> <schema.org/name> <schema.org/position> isa:<schema.org/ListItem>|6.28831774E8|
|                                                                                          <schema.org/itemListElement> isa:<schema.org/BreadcrumbList>|5.90033241E8|
|   

In [3]:
# calcule le count, l'average et le coverage d'un type de données
import json
import os
import re
def getCSETofType(data, pred, saveloc):
    
    # Danger injection SQL
    pred = pred.replace("'", "\\'")
    sets = spark.sql(f"SELECT pset, count FROM {data} WHERE pset LIKE '%{pred}%'")
    sets = sets.withColumn("pset", f.split(sets["pset"], " "))
    jsonsets = sets.toJSON().collect()
    
    pattern = r'isa:<schema\.org/(\w+)>'
    match = re.search(pattern, pred)
    typeN = match.group(1)
    
    if not os.path.exists(f"test-out/{saveloc}"):
        os.makedirs(f"test-out/{saveloc}")

    file_path = os.path.join(f"test-out/{saveloc}", f"{typeN}.json")
    
    # Create a list to hold individual JSON objects
    json_objects = []

    for json_str in jsonsets:
        # Load each JSON string as a Python dictionary
        json_obj = json.loads(json_str)
        json_objects.append(json_obj)
    
    with open(file_path, "w") as outfile:
        json.dump(json_objects, outfile, indent=4)
    
    

In [4]:
from pyspark.sql import Row

type_name_list = 

print(type_name_list)

for pred in type_name_list:
    print(pred)

    getCSETofType("CSET_avant", pred, "new-cset-avant-per-type")
    getCSETofType("CSET_apres", pred, "new-cset-apres-per-type")
    

['isa:<schema.org/Recipe>']
isa:<schema.org/Recipe>
